# Gemini SDK 기본 이해

**학습 목표**
- `google-genai` SDK 구조 파악
- `types` 모듈의 역할과 주요 클래스 이해
- Client, GenerateContentConfig 등 핵심 객체 이해

**참고 자료**
- [Google Gen AI Python SDK](https://github.com/googleapis/python-genai)
- [SDK API 문서](https://googleapis.github.io/python-genai/)

---
## 1. 환경 설정

In [ ]:
# SDK 설치 (최초 1회)
# !uv pip install google-genai python-dotenv

In [ ]:
# 필수 import
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types

# .env 파일에서 API 키 로드
load_dotenv()

# API 키 확인
api_key = os.getenv("GEMINI_API_KEY")
if api_key:
    print(f"✅ API 키 로드 완료: {api_key[:10]}...")
else:
    print("❌ GEMINI_API_KEY가 .env에 없습니다")

---
## 2. Client 객체

### `genai.Client()` - SDK의 진입점

모든 Gemini API 호출의 시작점입니다. `requests.Session()`이나 DB 커넥션과 유사한 역할을 합니다.

**파라미터:**
- `api_key` (str, 선택): Gemini API 키. 환경변수 `GEMINI_API_KEY`로도 자동 인식

**반환:** `Client` 객체

**예시:** `client = genai.Client(api_key="your-key")`

In [ ]:
# Client 생성
client = genai.Client(api_key=api_key)

print(f"✅ Client 생성 완료")
print(f"📦 Client 타입: {type(client)}")

### `client.models.generate_content()` - 텍스트 생성

가장 기본적인 API 호출 메서드입니다. 프롬프트를 보내고 응답을 받습니다.

**파라미터:**
- `model` (str, 필수): 사용할 모델명 (예: `"gemini-2.5-flash"`)
- `contents` (str | list, 필수): 프롬프트 텍스트 또는 Content 객체 리스트
- `config` (GenerateContentConfig, 선택): 생성 설정 (온도, 토큰 수 등)

**반환:** `GenerateContentResponse` 객체

**예시:** `response = client.models.generate_content(model="gemini-2.5-flash", contents="안녕?")`

In [ ]:
# 기본 텍스트 생성
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Python의 장점을 3가지만 간단히 알려줘"
)

print("📝 응답:")
print(response.text)

In [ ]:
# response 객체 구조 탐색
print("🔍 response 객체의 주요 속성들:")
print(f"  - type: {type(response)}")
print(f"  - text: {response.text[:50]}...")
print(f"  - candidates 개수: {len(response.candidates)}")

---
## 3. types 모듈 해부

`google.genai.types`는 SDK에서 사용하는 모든 데이터 구조를 정의합니다.  
JavaScript의 TypeScript 타입 정의, Python의 Pydantic 모델과 유사한 역할입니다.

### `types.Content` - 메시지 컨테이너

하나의 대화 턴(turn)을 나타냅니다. OpenAI의 `{"role": "user", "content": "..."}`와 동일한 개념입니다.

**파라미터:**
- `role` (str, 필수): `"user"` 또는 `"model"`
- `parts` (list[Part], 필수): 메시지 내용 리스트

**예시:** `types.Content(role="user", parts=[types.Part(text="안녕")])`

In [ ]:
# Content 객체 생성
user_message = types.Content(
    role="user",
    parts=[types.Part(text="서울의 인구는?")]
)

print(f"📦 Content 구조:")
print(f"  - role: {user_message.role}")
print(f"  - parts: {user_message.parts}")

### `types.Part` - 메시지 조각

텍스트, 이미지, 함수 호출 결과 등 다양한 형태의 콘텐츠를 담습니다.

**주요 속성:**
- `text` (str): 텍스트 내용
- `function_call` (FunctionCall): 함수 호출 정보
- `function_response` (FunctionResponse): 함수 실행 결과

**예시:** `types.Part(text="안녕하세요")`

In [ ]:
# Part 객체 생성
text_part = types.Part(text="함수 호출을 배워봅시다")

print(f"📦 Part 구조:")
print(f"  - text: {text_part.text}")
print(f"  - function_call: {text_part.function_call}")

### `types.GenerateContentConfig` - 생성 설정

모델의 동작을 제어하는 설정 객체입니다. 온도, 최대 토큰, 도구 등을 지정합니다.

**주요 파라미터:**
- `temperature` (float, 0~2): 창의성 조절. 0=결정적, 1=기본, 2=창의적
- `max_output_tokens` (int): 최대 출력 토큰 수
- `tools` (list): Function Calling에 사용할 함수 목록
- `system_instruction` (str): 시스템 프롬프트

**예시:** `types.GenerateContentConfig(temperature=0.7, max_output_tokens=100)`

In [ ]:
# GenerateContentConfig 사용
config = types.GenerateContentConfig(
    temperature=0.3,  # 낮은 온도 = 더 결정적인 응답
    max_output_tokens=50,
    system_instruction="당신은 간결하게 답변하는 AI입니다. 한 문장으로만 답하세요."
)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Python이란?",
    config=config
)

print("📝 간결한 응답:")
print(response.text)

### Function Calling 관련 타입 미리보기

다음 노트북에서 자세히 다루지만, 핵심 타입을 미리 살펴봅니다.

| 타입 | 역할 |
|-----|------|
| `types.Tool` | 함수 선언들을 묶는 컨테이너 |
| `types.FunctionDeclaration` | 개별 함수의 스키마 정의 |
| `types.FunctionCall` | 모델이 호출하려는 함수 정보 |
| `types.FunctionResponse` | 함수 실행 결과를 모델에 전달 |

In [ ]:
# Function Calling 타입 미리보기 (실행은 다음 노트북에서)
print("🔧 Function Calling 관련 타입들:")
print(f"  - types.Tool: {types.Tool}")
print(f"  - types.FunctionDeclaration: {types.FunctionDeclaration}")
print(f"  - types.ToolConfig: {types.ToolConfig}")

---
## 4. 응답 객체 분석

In [ ]:
# 분석용 응답 생성
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="1+1은?"
)

print("📊 응답 객체 전체 구조:")
print(response)

### `response.candidates` - 응답 후보들

모델이 생성한 응답 후보 리스트입니다. 보통 1개이지만, 설정에 따라 여러 개가 될 수 있습니다.

**구조:**
- `candidates[0].content` → Content 객체
- `candidates[0].content.parts[0].text` → 실제 텍스트

In [ ]:
# candidates 구조 탐색
candidate = response.candidates[0]

print("📦 candidate 구조:")
print(f"  - content.role: {candidate.content.role}")
print(f"  - content.parts: {candidate.content.parts}")
print(f"  - finish_reason: {candidate.finish_reason}")

### `response.usage_metadata` - 토큰 사용량

API 호출에 사용된 토큰 수를 확인합니다. 비용 계산과 컨텍스트 관리에 중요합니다.

In [ ]:
# TODO(human): response.usage_metadata에서 토큰 정보 출력
# 목적: 토큰 사용량 확인 방법 학습
# 힌트: prompt_token_count, candidates_token_count, total_token_count 속성
# 예상 출력:
#   - 입력 토큰: N개
#   - 출력 토큰: N개
#   - 총 토큰: N개

usage = response.usage_metadata
# 여기에 코드 작성


---
## 5. 실습: Content 객체로 멀티턴 대화

In [ ]:
# 멀티턴 대화 예시 (제공됨)
contents = [
    types.Content(
        role="user",
        parts=[types.Part(text="내 이름은 준영이야")]
    )
]

# 첫 번째 응답
response1 = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=contents
)
print(f"🤖 응답 1: {response1.text}")

# 대화 기록에 추가
contents.append(response1.candidates[0].content)

In [ ]:
# TODO(human): 두 번째 질문 추가하고 응답 받기
# 목적: 멀티턴 대화에서 컨텍스트 유지 확인
# 힌트: contents 리스트에 새 Content 추가 후 generate_content 호출
# 예상: 모델이 "준영"이라는 이름을 기억하고 응답

# 두 번째 질문: "내 이름이 뭐라고 했지?"
# 여기에 코드 작성


---
## 6. 정리

### 핵심 요약

| 구성 요소 | 역할 | 비유 |
|----------|------|------|
| `genai.Client` | API 연결 관리 | DB 커넥션 |
| `types.Content` | 대화 턴 컨테이너 | 채팅 메시지 |
| `types.Part` | 콘텐츠 조각 | 메시지 내용물 |
| `types.GenerateContentConfig` | 생성 설정 | 모델 파라미터 |
| `response.candidates` | 응답 후보 | 생성된 답변들 |
| `response.usage_metadata` | 토큰 사용량 | 비용 정보 |

### 다음 단계
→ `02_gemini_ftn_calling_operate__basic.ipynb`에서 Function Calling 실습